In [ ]:
import larch as lx
import pandas as pd
from larch import P,X

import numpy as np
import xarray as xr
from pytest import approx

In [ ]:
raw = pd.read_csv(lx.example_file('swissmetro.csv.gz'))
raw['SM_COST'] = raw['SM_CO'] * (raw["GA"]==0) 
raw['TRAIN_COST'] = raw.eval("TRAIN_CO * (GA == 0)") 
raw['TRAIN_COST_SCALED'] = raw['TRAIN_COST'] / 100
raw['TRAIN_TT_SCALED'] = raw['TRAIN_TT'] / 100

raw['SM_COST_SCALED'] = raw.eval('SM_COST / 100')
raw['SM_TT_SCALED'] = raw['SM_TT'] / 100

raw['CAR_CO_SCALED'] = raw['CAR_CO'] / 100
raw['CAR_TT_SCALED'] = raw['CAR_TT'] / 100
raw['CAR_AV_SP'] = raw.eval("CAR_AV * (SP!=0)")
raw['TRAIN_AV_SP'] = raw.eval("TRAIN_AV * (SP!=0)")

raw['keep'] = raw.eval("PURPOSE in (1,3) and CHOICE != 0")

In [ ]:
raw

In [ ]:
data = lx.Dataset.construct.from_idco(raw).dc.query_cases('keep')

In [ ]:
m1 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m1.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m1.choice_co_code = 'CHOICE'

m1.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_COST_SCALED") * P("B_COST")
m1.utility_co[2] = X("SM_COST_SCALED") * P("B_COST")
m1.utility_co[3] = P("ASC_CAR") + X("CAR_CO_SCALED") * P("B_COST")

In [ ]:
m1.parameters

In [ ]:
m2 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m2.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m2.choice_co_code = 'CHOICE'

m2.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_TT_SCALED") * P("B_TIME") + X("TRAIN_COST_SCALED") * P("B_COST")
m2.utility_co[2] = X("SM_TT_SCALED") * P("B_TIME") + X("SM_COST_SCALED") * P("B_COST")
m2.utility_co[3] = P("ASC_CAR") + X("CAR_TT_SCALED") * P("B_TIME") + X("CAR_CO_SCALED") * P("B_COST")

In [ ]:
m3 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m3.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m3.choice_co_code = 'CHOICE'

m3.utility_co[1] = X("TRAIN_COST_SCALED") * P("Z_COST")
m3.utility_co[2] = X("SM_COST_SCALED") * P("Z_COST")
m3.utility_co[3] = X("CAR_CO_SCALED") * P("Z_COST")

m3.groupid = 'ID'

In [ ]:
import jax.numpy as jnp

In [ ]:
mk = lx.Model(
    data.dc.set_altids([101, 102, 103])
)
mk.utility_co[102] = P("W_OTHER")
mk.utility_co[103] = P("W_COST")

In [ ]:
b = lx.LatentClass(
    mk, 
    {101:m1, 102:m2, 103:m3}, 
    datatree=data.dc.set_altids([1,2,3]),
    groupid="ID",
)

In [ ]:
b

In [ ]:
b.lock(Z_COST=-10000)

In [ ]:
b.reflow_data_arrays()

In [ ]:
b.groupid

In [ ]:
b.jax_loglike(b.pvals)

In [ ]:
b.jax_loglike_casewise(b.pvals)

In [ ]:
b.jax_probability(b.pvals)

In [ ]:
b.pvals

In [ ]:
assert b.jax_loglike(b.pvals) == approx(-6867.245, rel=1e-4)

In [ ]:
b.jax_loglike(b.pvals)

In [ ]:
b.jax_d_loglike(b.pvals) # 5.25s

In [ ]:
b.jax_d_loglike(b.pvals) # 5.25s

In [ ]:
result = b.jax_maximize_loglike()

In [ ]:
result

In [ ]:
assert result.loglike == approx(-4474.478515625, rel=1e-5)

In [ ]:
result.loglike

In [ ]:
b.parameters.to_dataframe()

In [ ]:
se, hess, ihess = b.jax_param_cov(b.pvals)

In [ ]:
b.parameters.to_dataframe()

In [ ]:
assert b.pstderr == approx(np.array([ 
    0.048158,  0.069796,  0.069555,  0.106282,  0.161079,  0.11945 ,  0.
]), rel=5e-3)

In [ ]:
b.pstderr

In [ ]:
assert b.pvals == approx(np.array([ 
    6.079781e-02, -9.362056e-01, -1.159657e+00, -3.095285e+00, -7.734768e-01,  1.155985e+00, -1.000000e+04
]), rel=5e-3)

In [ ]:
b.pvals

In [ ]:
ds = xr.Dataset()
ds = ds.dc.set_altids([1,2,3,4])
ds.dc.altids()

In [ ]:
ds.dc.n_alts

In [ ]:
ds = ds.dc.set_altids([7,8,9], dim_name='A')


In [ ]:
ds.dc.altids()